<a href="https://colab.research.google.com/github/TK-Problem/Python-mokymai/blob/master/Scripts/cvonline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title Importuoti paketus
#@markdown gali užtrukti iki 2 minučių
%%time
# playwright biblioteka naudojama importuoti html kodą
!pip install playwright==1.25.00
!playwright install-deps
!playwright install webkit
!pip install nest_asyncio==1.5.6

# playwright veikia TIK asyncio režimu
import nest_asyncio
nest_asyncio.apply()
import asyncio

# importuoti playwright versiją
from playwright.async_api import async_playwright

# bsė naudojama iš HTML ištraukti reikiamą informaciją
from bs4 import BeautifulSoup

# kartais reikia palaikyti kurį laiką programą veikiančią
import time

# paketas reikalingas piešti interaktyviems grafikams
import plotly.express as px
import plotly.io as pio

# nustatyti temą
pio.templates.default = "seaborn"

# paketai dirbti su skaičiais ir duomenimis
import pandas as pd
import numpy as np

# clear output komanda naudojama išvalyti informacijai
from IPython.display import clear_output
clear_output()

CPU times: user 830 ms, sys: 98.6 ms, total: 929 ms
Wall time: 16.6 s


# Duomenų atsisiuntimas

Funkcijos `cvonline` veikimo žingsniai:

1. sukuria `playwright` webdriver'į (webkit),
2. sukuria netikrą `user_agent`, kad svetainė tave laikytų tikru varotoju,
3. sugeneruoji `cvonline.lt` puslapio URL kartu su raktažodžių (keyword),
4. paspaudžia ant pop-up ir cookie mygtukų,
5. palaukia prevenciškai 2 sekundes,
6. atsisiunčia HTML kodą,
7. perkelia HTML kodą į `BeautifulSoup` objektą,
8. iteruojame per eilutes ir išsitraukiame reikiamą informaciją,
9. duomenis sukelia į `pandas` DataFrame objektą ir jį grąžina.

In [2]:
#@title `cvonline` funkcija
async def cvonline(keyword="python"):
    """
    This function returns all available job listings based on search keyword.
    Inputs:
      keyword (str)
    Output:
      returns pandas DataFrame
    """
    async with async_playwright() as p:

        # create webdriver/webkit
        browser = await p.webkit.launch()

        # create user agent for the webdriver
        user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:95.0) Gecko/20100101 Firefox/95.0'

        # create new page, i.e. new table in your browser
        page = await browser.new_page(user_agent=user_agent)

        # generate URL with a keyword
        url = f"https://cvonline.lt/lt/search?limit=1000&offset=0&keywords%5B0%5D={keyword}&fuzzy=true&suitableForRefugees=false&isHourlySalary=false&isRemoteWork=false&isQuickApply=false"

        # visit page, make timeout to 90 sec, i.e. 90 000 ms
        await page.goto(url, timeout=90_000)

        # click on pop-up window
        await page.click("//button[@class='jsx-4189752321 close-modal-button']")

        # click cookie button
        await page.click("//button[@class='cookie-consent-button']")

        # imlicit wait
        time.sleep(2)

        # get page html contents
        page_source = await page.content()

        # convert to bs4 object
        soup = BeautifulSoup(page_source, "lxml")

        # find all rows (ul - unordered list, li - list item)
        rows = soup.find("ul", {"class": "jsx-1871295890 jsx-78775730 vacancies-list"}).find_all("li")

        # create tmp. list to store data
        lst = list()

        # iterate over all rows
        for row in rows:
          # find all <a> tags
          for a in row.find_all('a', href=True):
            # condition to find employr info
            if "employer" in a['href']:
              # get element's text
              employer = a.text

          # get all row contents
          _contents = row.find_all("span")

          # get specific info about job title and job location
          job_title = _contents[0].text
          job_location = _contents[2].text

          # get start day (the date job was created)
          for c in _contents[4:]:
            # if it started
            if "Paskelbta" in c.text:
              # get text value
              offer_started = c.text.split("Baigiasi")[0]
            # if job offered is closed
            if "Baigiasi" in c.text:
              # get text value
              offer_ends = c.text

          # get salary value
          salary = row.find('span', {'class': 'jsx-3024910437 vacancy-item__salary-label'})

          # convert salary to text
          if salary:
            salary = salary.text
          else:
            salary = ''

          # add data to temp. list
          lst.append([employer, job_title, job_location, offer_started, offer_ends, salary])

        # save image to your enviroment (for debuging)
        # one can close this line
        await page.screenshot(path="cvonline_status.png")

        # close webkit
        await browser.close()

        # return pandas DataFrame
        return pd.DataFrame(lst, columns = ['Employer', 'JobTitle', "Location", "Offered", "AddEnds", "Salary"])

In [3]:
#@title Atsisiųsti duomenis
# paieškos žodis
keyword = 'python' # @param {type:"string"}

# iškviečiame funkciją ir išsaugome duomenis ir atvaizduojame pirmus 5 skelbimus
df_cvonline = asyncio.run(cvonline(keyword))

# parašyti kiek rado skelbimų
print(f"Rado {len(df_cvonline)} skelbimų.")

df_cvonline.head()

Rado 49 skelbimų.


,Employer,JobTitle,Location,Offered,AddEnds,Salary
0,"Quadigi, UAB",Test Automation Engineer (Python/C#),,Paskelbta prieš 11 dienų,Baigiasi: 2023.11.02,€ 3000 – 5500
1,"Planner5D, UAB",Python developer in AI team,,Paskelbta prieš 10 dienų,Baigiasi: 2023.11.03,€ 4500 – 7500
2,Wargaming,Python Software Engineer,,Paskelbta prieš 3 dienas,Baigiasi: 2023.11.10,€ 4300 – 5800
3,"Cybercare, UAB",AI Developer,,Paskelbta prieš 9 dienas,Baigiasi: 2023.11.04,€ 3300 – 6000
4,"QDev Technologies, UAB",Test Automation Engineer,,Paskelbta prieš 11 dienų,Baigiasi: 2023.11.02,€ 2000 – 4300


# Duomenų apdorojimas

Duomenis būtina sutvarkyti prieš pradedant analizuoti. Visa tai atlieka `clean_num_cols` funkcija.

Atlyginimo stulpelis `salary` turi keletą tipų reikšmių:

* vieni atlyginimai parašyti per ruožą, pvz. € 3300 – 4000. Tokiu atveju, reikia ištraukti minimalią ir maksimalią atlyginimo vertes, panaikinti euro simbolį.
* kiti skelbimai neskelbia atlygimų, tiesiog rašo "TOP Darbdavys". Tokius įrašus reikia paversti NaN vertėmis.
* yra atlyginimų, kur rašo valandinį, pvz. € 6/h, tokiu šį atlyginimą paversti į mėnesinį (22 darbo dienos po 8 valandas).

Galiausiai atlyginimai yra sunormuojami į vidurkį tarp minimalaus ir maksimalaus siūlomo varianto.

In [4]:
#@title Sutvarkyti skaitinius duomenis
def clean_num_cols(df):
  """
  Formats salary columns
  Input:
    df - pandas DataFrame
  Output:
    pandas DataFrame
  """
  # clean empty salaries (the ones with Top darbdavys)
  df.Salary = df.Salary.apply(lambda x: "" if "TOP Darbdavys" in x else x)

  # if there is salary range, e.g. x - y, then extract min and max values
  df['SalaryMin'] = df.Salary.apply(lambda x: x.split(" – ")[0][2:] if " – " in x else x)
  df['SalaryMax'] = df.Salary.apply(lambda x: x.split(" – ")[1] if " – " in x else x)

  # remove euro sign
  df['SalaryMin'] = df['SalaryMin'].str.replace("€", "")
  df['SalaryMax'] = df['SalaryMax'].str.replace("€", "")

  # assume that each month has 22 working days with 8 hours a day
  # approximate hourly wages to monthly
  # condition to select rows with hourly salary
  cond_1 = df.Salary.apply(lambda x: "/h" in x)
  # condition with are salary range
  cond_2 = df.Salary.apply(lambda x: " – " in x)

  # convert hourly data to month salaries with single hourply pay
  df.loc[cond_1 & ~cond_2, 'SalaryMin'] = df.loc[cond_1 & ~cond_2, 'Salary'].apply(lambda x: float(x.split("/h")[0].replace("€", "")) * 22 * 8)
  df.loc[cond_1 & ~cond_2, 'SalaryMax'] = df.loc[cond_1 & ~cond_2, 'Salary'].apply(lambda x: float(x.split("/h")[0].replace("€", "")) * 22 * 8)

  # convert hourly data to month salaries with hourly pay in range
  df.loc[cond_1 & cond_2, 'SalaryMin'] = df.loc[cond_1 & cond_2, 'Salary'].apply(lambda x: float(x.split(" – ")[0].replace("€", "")) * 22 * 8)
  df.loc[cond_1 & cond_2, 'SalaryMax'] = df.loc[cond_1 & cond_2, 'Salary'].apply(lambda x: float(x.split(" – ")[1][:-2]) * 22 * 8)

  # convert missing salaries to NaNs
  df.loc[df.SalaryMin == '', 'SalaryMin'] = np.nan
  df.loc[df.SalaryMax == '', 'SalaryMax'] = np.nan

  # convert to floats
  df['SalaryMin'] = df['SalaryMin'].astype(float)
  df['SalaryMax'] = df['SalaryMax'].astype(float)

  # calculate average salary
  df['SalaryMean'] = (df['SalaryMin'] + df['SalaryMax']) / 2

  # return cleaned DataFrame
  return df

# clean numerical values
df_c = clean_num_cols(df_cvonline.copy())

# drop adds without salary
df_c = df_c.dropna()
df_c.head()

,Employer,JobTitle,Location,Offered,AddEnds,Salary,SalaryMin,SalaryMax,SalaryMean
0,"Quadigi, UAB",Test Automation Engineer (Python/C#),,Paskelbta prieš 11 dienų,Baigiasi: 2023.11.02,€ 3000 – 5500,3000.0,5500.0,4250.0
1,"Planner5D, UAB",Python developer in AI team,,Paskelbta prieš 10 dienų,Baigiasi: 2023.11.03,€ 4500 – 7500,4500.0,7500.0,6000.0
2,Wargaming,Python Software Engineer,,Paskelbta prieš 3 dienas,Baigiasi: 2023.11.10,€ 4300 – 5800,4300.0,5800.0,5050.0
3,"Cybercare, UAB",AI Developer,,Paskelbta prieš 9 dienas,Baigiasi: 2023.11.04,€ 3300 – 6000,3300.0,6000.0,4650.0
4,"QDev Technologies, UAB",Test Automation Engineer,,Paskelbta prieš 11 dienų,Baigiasi: 2023.11.02,€ 2000 – 4300,2000.0,4300.0,3150.0


# Įžvalgos

Keletas klausimų į kurios galima atsakyti tiek vizualiai tiek skaičiais.

In [5]:
#@title Vidutinis atlyginimas

# atspausindit atsakymą
print(f'Pagal pieškos žodį "{keyword}" buvo {len(df_c)} skelbimai vidutiškai siūlo {df_c.SalaryMean.mean():.0f} € atlyginimą')

Pagal pieškos žodį "python" buvo 49 skelbimai vidutiškai siūlo 3857 € atlyginimą


In [6]:
#@title Top skelbimai su didžiausiais atlyginimais

# top N dižiausius atlyginimus turintys skelbimai
N = 10 # @param {type:"integer"}
cols = ['Employer', 'JobTitle', 'AddEnds', 'Salary', 'SalaryMean']

# sort and return N largest
df_c.sort_values(by="SalaryMean").tail(N)[cols].iloc[::-1]

,Employer,JobTitle,AddEnds,Salary,SalaryMean
21,"QDev Technologies, UAB",Software Configuration Manager (Cloud),Baigiasi: 2023.11.10,€ 5000 – 7400,6200.0
1,"Planner5D, UAB",Python developer in AI team,Baigiasi: 2023.11.03,€ 4500 – 7500,6000.0
28,"Integre Trans, UAB",Product owner,Baigiasi: 2023.10.22,€ 4463 – 6612,5537.5
13,Adform,Senior/ Lead Data Scientist,Baigiasi: 2023.10.25,€ 5500,5500.0
31,Danske Bank Lithuania,Senior Cloud Specialist: Kubernetes and Linux,Baigiasi: 2023.10.27,€ 4320 – 6480,5400.0
29,"Revel Systems, UAB",Swift Developer,Baigiasi: 2023.10.21,€ 5200,5200.0
2,Wargaming,Python Software Engineer,Baigiasi: 2023.11.10,€ 4300 – 5800,5050.0
27,"Yukon Advanced Optics Worldwide, UAB",Data Scientist,Baigiasi: 2023.10.26,€ 4000 – 6000,5000.0
22,"Quadigi, UAB",Lead Software Engineer C++,Baigiasi: 2023.11.03,€ 5000,5000.0
24,Danske Bank Lithuania,Experienced Data Engineer in Super AI,Baigiasi: 2023.10.29,€ 4000 – 6000,5000.0


In [7]:
#@title Top skelbimai su didžiausiais atlyginimais pagal unikalų darbdavį

# top N dižiausius atlyginimus turintys skelbimai
N = 10 # @param {type:"integer"}
cols = ['Employer', 'JobTitle', 'AddEnds', 'Salary', 'SalaryMean']

# sort and return N largest
_ = df_c.sort_values(by="SalaryMean").drop_duplicates(subset='Employer', keep="last")
_.tail(N)[cols].iloc[::-1]

,Employer,JobTitle,AddEnds,Salary,SalaryMean
21,"QDev Technologies, UAB",Software Configuration Manager (Cloud),Baigiasi: 2023.11.10,€ 5000 – 7400,6200.0
1,"Planner5D, UAB",Python developer in AI team,Baigiasi: 2023.11.03,€ 4500 – 7500,6000.0
28,"Integre Trans, UAB",Product owner,Baigiasi: 2023.10.22,€ 4463 – 6612,5537.5
13,Adform,Senior/ Lead Data Scientist,Baigiasi: 2023.10.25,€ 5500,5500.0
31,Danske Bank Lithuania,Senior Cloud Specialist: Kubernetes and Linux,Baigiasi: 2023.10.27,€ 4320 – 6480,5400.0
29,"Revel Systems, UAB",Swift Developer,Baigiasi: 2023.10.21,€ 5200,5200.0
2,Wargaming,Python Software Engineer,Baigiasi: 2023.11.10,€ 4300 – 5800,5050.0
27,"Yukon Advanced Optics Worldwide, UAB",Data Scientist,Baigiasi: 2023.10.26,€ 4000 – 6000,5000.0
22,"Quadigi, UAB",Lead Software Engineer C++,Baigiasi: 2023.11.03,€ 5000,5000.0
17,Alliance for Recruitment,Senior Data Engineer,Baigiasi: 2023.10.19,€ 3855 – 5782,4818.5


# Išsaugoti duomenis

Atkomentuoti eilutes su `CTR + /` ir išsaugoti norimu formatu. Failo pavadinimas sugenruojams:

* `cv_online_Y_X.csv` arba `cv_online_Y_X.xlsx`, kur `X` yra raktažodis, o `Y` yra data, kad buvo paleistas kodas. Jei `X` buvo iš dviejų žodžių, pvz. `duomenų analitikas`, tarpai ` ` yra paverčiami `_`.

Norint atsisiųsti duomenis lokaliai paspauskite ant dešinėje pusė esančios "Files" ikonos ir atsisiųskite norimą failą. Jei to nepadarysite, failai bus ištrinti uždarius google colab.

![image info](https://i.stack.imgur.com/mYWnb.png)




In [8]:
#@title Kodo pavyzdžiai

# generate data for today (use it for file name)
_date = pd.Timestamp.now().strftime("%Y_%m_%d")

# excel, remove comments if you want to save data
# df_c.to_excel(f"cv_online_{keyword}_{_date}.xlsx", index = False)
# .csv, remove comments if you want to save data
# df_c.to_csv(f"cv_online_{keyword}_{_date}.csv", index = False)

# Darbdavių siūlomų atlyginimų analizė

Tikslas yra:

* nuskaityti darbo skelbimus pagal kelis raktažodžius,
* sujungti lenteles,
* išfiltruoti pasikartojančius skelbimus,
* palyginti darbdavių, kurie yra įkėlę bent 10 skelbimų, siūlomus atlyginimų rėžius.

In [9]:
%%time
#@title Nuskaityti duomenis

# list of companies we are interested in
keywords = ['bankas', 'vadyba', 'administracija', 'IT']

# create empty DataFrame to store all data
df_all = pd.DataFrame()

# iterate over keywords
for i, keyword in enumerate(keywords):
  # read cvonline data
  _df = asyncio.run(cvonline(keyword.lower()))

  # clean numerical values
  _df = clean_num_cols(_df)

  # drop adds without salary
  _df = _df.dropna()

  # add data to main DataFrame
  df_all = pd.concat([df_all, _df])

  print(f'Duomenys nuskaityti pagal "{keyword}" raktažodį ({len(_df)} skelbimai). ({i+1}/{len(keywords)})')

Duomenys nuskaityti pagal "bankas" raktažodį (251 skelbimai). (1/4)
Duomenys nuskaityti pagal "vadyba" raktažodį (530 skelbimai). (2/4)
Duomenys nuskaityti pagal "administracija" raktažodį (11 skelbimai). (3/4)
Duomenys nuskaityti pagal "IT" raktažodį (953 skelbimai). (4/4)
CPU times: user 6.56 s, sys: 286 ms, total: 6.84 s
Wall time: 1min 38s


In [10]:
#@title Sutvarkyti duomenis
# exclude some columns so that duplicate adds could be removed
df_all_c = df_all[['Employer', 'JobTitle', 'Salary', 'SalaryMin', 'SalaryMax', 'SalaryMean']]

# remove duplicates
df_all_c = df_all_c.drop_duplicates().reset_index(drop=True)

# clean string columns
df_all_c.Employer = df_all_c.Employer.str.strip()
df_all_c.JobTitle = df_all_c.JobTitle.str.strip()

df_all_c.head()

,Employer,JobTitle,Salary,SalaryMin,SalaryMax,SalaryMean
0,"Šiaulių bankas, AB",Klientų aptarnavimo vadybininkas (-ė),€ 1080 – 1620,1080.0,1620.0,1350.0
1,"Šiaulių bankas, AB",Klientų aptarnavimo vadybininkas (-ė) PC Akrop...,€ 1140 – 1720,1140.0,1720.0,1430.0
2,"Šiaulių bankas, AB",Asistentas (-ė) Lizingo departamentas,€ 1140 – 1720,1140.0,1720.0,1430.0
3,"Šiaulių bankas, AB",Sukčiavimo prevencijos grupės specialistas (-ė),€ 1360 – 2040,1360.0,2040.0,1700.0
4,"Šiaulių bankas, AB",Specialistas (-ė) Skaitmeninių kanalų grupėje,€ 1360 – 2040,1360.0,2040.0,1700.0


In [11]:
#@title Didžiausi darbdaviai
#@markdown Pagal skelbimų skaičių.

TOP_N = 10 #@param {type: "number"}

# get employers with largest number of job adds
df_largest_emp = df_all_c.groupby(['Employer'])['JobTitle'].count()

# output results in DataFrame
_df = pd.DataFrame(df_largest_emp.sort_values(ascending=True))
_df.columns = ['No. of job postings']
_df.tail(TOP_N)[::-1]

,No. of job postings
Employer,
BIURO,121
Alliance for Recruitment,106
Danske Bank Lithuania,106
Ignitis grupė,90
"Lietuvos geležinkeliai, AB",57
MAXIMA,57
"People Link, UAB",45
"Lidl Lietuva, UAB",42
Noriu personalo sprendimų grupė,31


In [12]:
#@title Siūlomų darbo skelbimų atlyginimų rėžiai
#@markdown TOP 12 darbdavių

# get largest employer names
_names = _df.index[-12:]

# quick figure
fig = px.box(df_all_c.loc[df_all_c.Employer.isin(_names)],
             y="SalaryMean", facet_col="Employer", color="Employer",
             hover_name = 'JobTitle', hover_data = ['Salary'],
             boxmode="overlay", points='all', facet_col_wrap = 4)
fig.update_layout(showlegend=False)
fig.show()

# Darbdavio skelbimų analizė

In [13]:
#@title Stačiakampė diagrama

employer = "BIURO" #@param {type: "string"}

# quick figure
fig = px.box(df_all.loc[df_all.Employer == employer],
             y="SalaryMean", hover_name = 'JobTitle',
             hover_data = ['Salary', 'Location'], boxmode="overlay", points='all')
fig.update_layout(showlegend=False)
fig.show()